In [3]:
# So ... each agent also has a type ideally

#Thought: with the type, we could set it within the agent itself.
#That way only need to pass the type?
#
#I guess that agents don't have access to the model?

In [4]:
# https://stackoverflow.com/questions/19620498/how-to-create-a-utility-class-correctly

# Okay... so... 
We have agents...

Suspicious ones
Not suspicious ones


Transactions should have:


We also want to export the network of transactions

So... the data we want are... 

sender - receiver - amount - timestamp

parameters:

num of agents,
avg txn hr -> so at any given point, the likelihood of making a transaction

avg txn value
distribution of p per agent type 
starting balance




In [1]:
# export some... network graphs i guess....
# also want to make some classifieirs

In [2]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns
import pandas as pd

import scipy.stats
import datetime
import matplotlib.pyplot as plt

In [3]:
# work
rng = np.random.default_rng(42)
rng.random()
rng.normal(loc=0, scale=1, size=1)

array([-1.03998411])

# utility functions

In [28]:
DEBUG = False

def setup_p_txns(total_steps):
    # Mean at 0
    max_std_devs = 5 # 

    if DEBUG:
        # Note: scipy norm has stddev = 1, mean of 0
        # Below shows why we pick max_std_devs = 5
        p_txn = scipy.integrate.quad(
                scipy.stats.norm.pdf, -5, 5)[0] # quad() Returns: value, error
        print('In total we will capture this amount of the gaussian: ', p_txn)
    
    txn_probabilities = []

    std_dev_per_step = 2 * max_std_devs / total_steps

    for step in range(total_steps):
        p_txn = \
            scipy.integrate.quad(
                scipy.stats.norm.pdf, 
                step * std_dev_per_step, (step+1) * std_dev_per_step
            ) # returns: (integration) value, error
        txn_probabilities.append(p_txn[0])
    if DEBUG:
        txn_probabilities[0] = 1 # set p_txn to 1 to always transact
        txn_probabilities[-1] = 1
        txn_probabilities[1:-1] = 0
    return txn_probabilities

# agent

In [192]:
class BankAgent(ap.Agent):
    def setup(self):
        self.txn_amt_rng = None 
        self.txn_probabilities = None 
        self.type = None 
        self.acct_balance = None 
        self.mean_txns = 4

        self.txns = pd.DataFrame(
                columns=['timestep', 'txn_type', 'sender_id', 'sender_type', 
                 'receiver_id', 'receiver_type', 'amount', 'acct_balance',] 
        )
        self.txn_amts = None

    def setup_txn_amts(self, mean=100, stddev=10, total_steps=0):
        self.txn_amts = self.txn_amt_rng.normal(
            loc=mean, scale=stddev, 
            size=total_steps)

    def transact(self, timestep):
        p_txn = self.txn_probabilities[timestep-1] * self.mean_txns
        if np.random.random() < p_txn and self.acct_balance > 0:
            # randomly choose partner
            partner = self.model.agents.random()
            # randomly choose amount
            amount = self.txn_amts[timestep]

            # calculations
            partner.acct_balance += amount
            self.acct_balance -= amount

            print(
                f' Transaction @ step: {timestep} with ptxn: {p_txn:.2f}, '
                f'amt: {amount:.2f} from: {self.id}->{partner.id}, '
                f'({self.type} to {partner.type}), new balance: {self.acct_balance:.1f}'
            )
            
            # record
            my_partner = partner.to_list()[0]

            # Note that txn_type is redundant info (e.g. can be derived from +/- of amount)
            self.txns.loc[len(self.txns.index)] = [
                timestep, 'send', self.id, self.type, my_partner.id, my_partner.type,
                -amount, self.acct_balance]

            #print('PARTNER TXNS', partner.to_list()[0].txns)
            my_partner.txns.loc[len(my_partner.txns.index)] = [
                timestep, 'receive', self.id, self.type, my_partner.id, my_partner.type,
                amount, my_partner.acct_balance]
            #.loc[len(partner.txns.index)] = [
            #    timestep, *partner.id, partner.type, self.id, self.type,
            #    amount, partner.acct_balance]

# parameters

In [193]:
        #self.txn_value_rng = p.rng
        #self.txn_probabilities = p.txn_probabilities
        ##self.p.type = 'default'
        #self.type = self.p.type
        #self.acct_balance = self.p.init_acct_balance

MEAN_TXN_HRS = {'normal':14, 
                'suspicious':22}
MINS_PER_STEP = 15

MEAN_TXN_AMOUNTS = {'normal':250, 
                'suspicious':50}

NUM_AGENTS_PER_TYPE = {'normal': 10,
    'suspicious': 2
}

parameters = {
    'num_agents_per_type': NUM_AGENTS_PER_TYPE,
    'mean_txn_hrs': MEAN_TXN_HRS,
    'mean_txn_amounts ': MEAN_TXN_AMOUNTS,
    'mean_txns': 4, # avg num txns each agent makes
    'starting_balance': 100,
    'seed': 42,
    'mins_per_step' : MINS_PER_STEP, # 1 hr
    'steps': int(24 * (60/MINS_PER_STEP)), # 24 hours * steps per hr
}

# For plotting
def timestep_to_time(timestep):
    date_and_time = datetime.datetime(2022, 10, 31, 0, 0, 0)
    time_elapsed = timestep * (parameters['mins_per_step'])
    time_change = datetime.timedelta(minutes=time_elapsed)
    new_time = date_and_time + time_change
    return new_time

In [194]:
a = {'asdf':1, 'dfg':2}
a.values()

dict_values([1, 2])

In [195]:
np.random.default_rng(42).random(10).round(3) * 1000

array([774., 439., 859., 697.,  94., 976., 761., 786., 128., 450.])

In [196]:
class BankModel(ap.Model):
    def setup(self):
        self.p_txns = setup_p_txns(self.p.steps) 
        num_agents = sum(self.p.num_agents_per_type.values())
        
        # Setup up rng to generate seeds for rngs for agents
        rng = np.random.default_rng(self.p.seed)
        agent_rng_seeds = rng.random(num_agents).round(3) * 1000
        agent_rng_seeds = np.array(agent_rng_seeds, dtype=int)

        self.agents = ap.AgentList(self) 
        for type in ['normal', 'suspicious']:

            hrs_mean = self.p.mean_txn_hrs[type]
            shift = hrs_mean * int(60 / self.p.mins_per_step)
            # shift in steps
            shifted = np.concatenate(
                (self.p_txns[-shift:], self.p_txns[:-shift]))
            agents = ap.AgentList(
                self, self.p.num_agents_per_type[type], BankAgent)

            agents.txn_probabilities = shifted
            agents.total_steps = self.p.steps 
            agents.acct_balance = 100
            agents.type = type

            self.agents += agents

        print('here are all the agetns: ')
        print(self.agents)
        print([(agent.id,agent.type) for agent in self.agents])

        # calcuations for txn values
        txn_amt_rngs = ap.AttrIter(
            [np.random.default_rng(seed) for seed in agent_rng_seeds])
        self.agents.txn_amt_rng = txn_amt_rngs

        for agent in self.agents:
            agent.setup_txn_amts(
                mean=20, stddev=5, total_steps=self.p.steps)
                #mean=100, stddev=10, total_steps=self.p.mean_txns * 5)
            # TODO: maybe I should make smaller table...? 
            # b ut then need to index into it, not by timestep
            # fundamental issue is ... wanting a same set of rngs for each agent
            # but cannot call at each timestep -- would get same value -- so instead pregenerate the values for all timesteps and then index into that
            #print(agent.txn_amts)
            #print(agent.id)
            #print(agent.type)


    def step(self):
        self.agents.transact(self.t)

    def update(self):
        #self.record(foo_bar)
        total_txns = 0
        for agent in self.agents:
            total_txns += agent.txns.shape[0]
        ('num txns (across all agents)', total_txns)

    def end(self):
        self.agents.record('txns')

In [197]:
np.random.default_rng(42).normal()

0.30471707975443135

In [198]:
model = BankModel(parameters)
results = model.run()

here are all the agetns: 
AgentList (12 objects)
[(1, 'normal'), (2, 'normal'), (3, 'normal'), (4, 'normal'), (5, 'normal'), (6, 'normal'), (7, 'normal'), (8, 'normal'), (9, 'normal'), (10, 'normal'), (11, 'suspicious'), (12, 'suspicious')]
Completed: 56 steps Transaction @ step: 57 with ptxn: 0.17, amt: 23.57 from: 3->[5], (normal to ['normal']), new balance: 76.4
Completed: 57 steps Transaction @ step: 58 with ptxn: 0.16, amt: 17.33 from: 10->[4], (normal to ['normal']), new balance: 82.7
Completed: 58 steps Transaction @ step: 59 with ptxn: 0.16, amt: 17.80 from: 6->[4], (normal to ['normal']), new balance: 82.2
Completed: 59 steps Transaction @ step: 60 with ptxn: 0.16, amt: 20.03 from: 7->[3], (normal to ['normal']), new balance: 80.0
 Transaction @ step: 60 with ptxn: 0.16, amt: 13.54 from: 8->[12], (normal to ['suspicious']), new balance: 86.5
 Transaction @ step: 60 with ptxn: 0.16, amt: 21.08 from: 10->[2], (normal to ['normal']), new balance: 61.6
Completed: 60 steps Transact

In [190]:
# For plotting
def timestep_to_time(timestep):
    date_and_time = datetime.datetime(2022, 10, 31, 0, 0, 0)
    time_elapsed = timestep * (parameters['mins_per_step'])
    time_change = datetime.timedelta(minutes=time_elapsed)
    new_time = date_and_time + time_change
    return new_time

In [199]:
for agent in model.agents:
    print(agent.txns)

   timestep txn_type  sender_id sender_type  receiver_id receiver_type  \
0        61     send          1      normal           11    suspicious   
1        62     send          1      normal            2        normal   
2        62  receive         10      normal            1        normal   
3        63     send          1      normal            1        normal   
4        63  receive          1      normal            1        normal   
5        67  receive          3      normal            1        normal   

      amount  acct_balance  
0 -16.819260     83.180740  
1 -19.496012     63.684728  
2  17.932182     81.616911  
3 -16.109051     81.616911  
4  16.109051     81.616911  
5  15.427996     97.044907  
   timestep txn_type  sender_id sender_type  receiver_id receiver_type  \
0        60  receive         10      normal            2        normal   
1        62  receive          1      normal            2        normal   
2        63  receive          7      normal            2

In [ ]:
# transaction amount, sender, receiver, timestamp